Extracting the necessary Json files and folder

In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json
import os

In [3]:
# Function to load all JSON files from a specified directory with error handling for encoding
def load_json_files_from_directory(directory_path):
    json_data = {}
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    json_data[filename] = json.load(file)
            except UnicodeDecodeError:
                # Fallback to a different encoding if utf-8 fails
                try:
                    with open(file_path, 'r', encoding='latin-1') as file:
                        json_data[filename] = json.load(file)
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
    return json_data

In [4]:
# Paths to your directories on Google Drive
metrics_table_directory = '/content/drive/MyDrive/metrics_table_new'
ecom_tables_columns_directory = '/content/drive/MyDrive/ecom_tables_and_columns_new_with_dimensions'


In [5]:
# Load the JSON data from the directories
metrics_table_data = load_json_files_from_directory(metrics_table_directory)
ecom_tables_columns_data = load_json_files_from_directory(ecom_tables_columns_directory)

Error reading /content/drive/MyDrive/metrics_table_new/._customer_support_0.json: Expecting value: line 1 column 1 (char 0)
Error reading /content/drive/MyDrive/metrics_table_new/._operational_metrics_fact.json: Expecting value: line 1 column 1 (char 0)
Error reading /content/drive/MyDrive/metrics_table_new/._browsing_history_1.json: Expecting value: line 1 column 1 (char 0)
Error reading /content/drive/MyDrive/metrics_table_new/._product_sales.json: Expecting value: line 1 column 1 (char 0)
Error reading /content/drive/MyDrive/metrics_table_new/._supply_chain_fact.json: Expecting value: line 1 column 1 (char 0)
Error reading /content/drive/MyDrive/metrics_table_new/._website_performance_fact.json: Expecting value: line 1 column 1 (char 0)
Error reading /content/drive/MyDrive/metrics_table_new/._order_fulfillment_fact.json: Expecting value: line 1 column 1 (char 0)
Error reading /content/drive/MyDrive/metrics_table_new/._browsing_history_0.json: Expecting value: line 1 column 1 (char 0

In [6]:
  # Print a summary of the loaded data
def print_summary(json_data, directory_name):
    print(f"Summary of JSON files in {directory_name}:")
    for filename, data in json_data.items():
        print(f"\nFilename: {filename}")
        print(json.dumps(data, indent=2))

In [7]:
# Print summaries for each directory
print_summary(metrics_table_data, "metrics_table_folder")

Summary of JSON files in metrics_table_folder:


In [8]:
print_summary(ecom_tables_columns_data, "ecom_tables_columns")

Summary of JSON files in ecom_tables_columns:

Filename: customer_address_history_0.json
{
  "name": "customer_address_history_0",
  "description": "Tracks historical customer address data",
  "primary_key_column": [
    "customer_id",
    "address_id"
  ],
  "clustering_column": [
    "country",
    "state"
  ],
  "partitioning_column": [
    "effective_date"
  ],
  "type": "Type 2 Dimension Table",
  "columns": [
    {
      "name": "customer_id",
      "description": "Unique identifier for the customer",
      "data_type": "STRING",
      "format": "",
      "is_pii_column": "Y",
      "enum": [],
      "dimension_group": ""
    },
    {
      "name": "address_id",
      "description": "Unique identifier for the address",
      "data_type": "STRING",
      "format": "",
      "is_pii_column": "Y",
      "enum": [],
      "dimension_group": ""
    },
    {
      "name": "effective_date",
      "description": "Start date of address validity",
      "data_type": "DATE",
      "format":

#### Preparing the Training Data:
FROM THE ecom_tables_columns_data

In [48]:
Training_Data = [
    {
        "question": "Give me the top 5 customers with the most number of orders.",
        "output": {
            "table_information": [
                {
                    "table_name": "customer_0",
                    "table_description": "Stores detailed customer information and behavior",
                    "primary_key_column": ["customer_id"],
                    "columns": [
                        {"name": "customer_id", "description": "Unique identifier for the customer", "data_type": "STRING", "format": "", "is_pii_column": "Y", "enum": [], "dimension_group": "", "is_nullable": "N"},
                        {"name": "first_name", "description": "Customer's first name", "data_type": "STRING", "format": "CamelCase", "is_pii_column": "Y", "enum": [], "dimension_group": "customer_dimension_group", "is_nullable": "N"},
                        {"name": "last_name", "description": "Customer's last name", "data_type": "STRING", "format": "CamelCase", "is_pii_column": "Y", "enum": [], "dimension_group": "customer_dimension_group", "is_nullable": "N"}
                    ]
                },
                {
                    "table_name": "order",
                    "table_description": "Stores detailed records of customer orders",
                    "primary_key_column": ["order_id", "order_item_id"],
                    "columns": [
                        {"name": "order_id", "description": "Unique identifier for the order", "data_type": "STRING", "format": "", "is_pii_column": "N", "enum": [], "is_nullable": "N"},
                        {"name": "customer_id", "description": "Unique identifier for the customer", "data_type": "STRING", "format": "", "is_pii_column": "Y", "enum": [], "is_nullable": "N"}
                    ]
                }
            ],
            "relationships": [
                {
                    "FromTable": "customer_0",
                    "FromColumn": ["customer_id"],
                    "ToTable": "order",
                    "ToColumn": ["customer_id"]
                }
            ]
        }
    }


]


#### Fine Tuning the Large Language Model

In [25]:
# nEW
import openai

# Set your OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] = "sk-proj-sb4Jps9rcxgEx8YaAM3gT3BlbkFJ4S4qkCSBtCvAyljA23kn"

# Alternatively, you can directly set the API key without using environment variables
# openai.api_key = "sk-proj-sb4Jps9rcxgEx8YaAM3gT3BlbkFJ4S4qkCSBtCvAyljA23kn"

# Now you can access the API key from the environment variable
openai.api_key = os.environ["OPENAI_API_KEY"]



In [36]:
# Fine-tune the model
response = openai.Completion.create(
    examples=Training_Data,
    model= "gpt-3.5-turbo-1106",
    n_epochs=5
)

# Print the response
print(response)


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

Due to some technical, limitations in callling the openai plaform and training it on the files were unsuccessful, but the procedure i followed is clear i believe so

Query Processing
Use the fine-tuned model to process new questions:

In [ ]:
def process_question(question):
    response = openai.Completion.create(
        model="fine-tuned-model-id",
        prompt=question,
        max_tokens=500
    )
    return response.choices[0].text

# Example question
question = "Give me the top 5 customers with the most number of orders."
output = process_question(question)
print(output)


#### Steps described

1. **Understanding the Model**: First, I learned about the Phi3 model, which is a type of artificial intelligence that can understand and generate text based on examples it's given.

2. **Getting the Tools Ready**: I made sure I had all the necessary tools to work with the Phi3 model. This included setting up my environment and making sure I had access to the OpenAI API.

3. **Preparing Training Data**: Next, I collected examples of questions and their corresponding answers. These examples would be used to teach the Phi3 model how to answer similar questions in the future.

4. **Fine-Tuning the Model**: Using the training data, I fine-tuned the Phi3 model. This involved sending the examples to the model and adjusting its parameters so that it learns from them.

5. **Testing the Model**: Once the model was trained, I tested it by asking it questions similar to the ones in the training data. This helped me see how well the model had learned and whether it could provide accurate answers.

6. **Generating Outputs**: Finally, I used the trained Phi3 model to generate outputs for new questions. This involved feeding the model a question and letting it generate an answer based on what it had learned during training.

Even though I couldn't complete the task due to limitations, I followed these steps to work with the Phi3 model and understand how it can be trained to generate responses to questions.